In [ ]:

"""
The following code simulates iVQAIC algorithm on QASM simulator.
iVQAIC algorithm is a variational quantum algorithm for inequality constrained semidefinite programs (SDPs) that we proposed in our paper.
Here, we use iVQAIC to solve random instances of an inequality constrained SDP.
The simulation is set up for N = 8, where N is the dimension of input Hermitian operators of an SDP.
Results for different N values can be obtained by modifying this parameter and making minor adjustments.
"""

In [ ]:
# Install required packages
!pip install qiskit qiskit-algorithms cvxpy matplotlib scipy numpy
!pip install -U qiskit-aer

In [ ]:
# Import necessary libraries
import cvxpy as cp
import numpy as np
import scipy.stats as stats
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from qiskit_aer import Aer
from qiskit_algorithms import VQE
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import Operator, SparsePauliOp
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator

In [ ]:
def sparse_rand_sym(n, density):
    """
    Generate a random symmetric sparse matrix.

    Args:
    n (int): Dimension of the matrix.
    density (float): Sparsity of the matrix.

    Returns:
    scipy.sparse.csr_matrix: A random symmetric sparse matrix.
    """
    rvs = stats.norm().rvs
    X = sparse.random(n, n, density=density, data_rvs=rvs)
    upper_X = sparse.triu(X)
    result = upper_X + upper_X.T - sparse.diags(X.diagonal())
    return result

# Constants of an SDP
# N: matrix dimension, M: number of constraints, R: trace upper bound
N, M, R = 8, 3, 10

# Generate a weakly constrained random sparse SDP
C = sparse_rand_sym(N, 0.1) # Objective function matrix
A = [sparse_rand_sym(N, 0.1) for _ in range(M)] # Constraint matrices
b = np.random.randn(M) # Constraint bounds

# Solve the SDP using CVXPY (classical solver for comparison)
X = cp.Variable((N,N), symmetric=True)
constraints = [X >> 0] + [cp.trace(A[i]@X) <= b[i] for i in range(M)] + [cp.trace(X) <= R]
prob = cp.Problem(cp.Maximize(cp.trace(C@X)), constraints)
prob.solve()
print("Optimal value:", prob.value)

In [ ]:
# Set up the quantum backend and estimator
backend = Aer.get_backend("aer_simulator")
estimator = Estimator()

def matrix_to_pauliop(matrix):
    """
    Convert a matrix to a Pauli operator representation suitable for quantum computation.

    Args:
    matrix (scipy.sparse.csr_matrix): Input matrix to convert.

    Returns:
    SparsePauliOp: Pauli operator representation of the input matrix.
    """
    # Convert sparse matrix to dense numpy array
    matrix_dense = matrix.toarray()

    # Get the next power of 2 for the matrix dimension
    n = matrix_dense.shape[0]
    next_power_of_2 = 2**int(np.ceil(np.log2(n)))

    # Pad the matrix with zeros to make it 2^n x 2^n
    padded_matrix = np.zeros((next_power_of_2, next_power_of_2), dtype=complex)
    padded_matrix[:n, :n] = matrix_dense

    # Create a Qiskit Operator
    op = Operator(padded_matrix)

    # Convert to SparsePauliOp
    return SparsePauliOp.from_operator(op)


# Convert C to SparsePauliOp
C_op = matrix_to_pauliop(C)

# Convert A matrices to SparsePauliOp
A_ops = [matrix_to_pauliop(A[i]) for i in range(M)]

In [ ]:
# iVQAIC algorithm parameters
learning_rate = 0.008
num_of_iterations = 54
diff_cost_func_iter_store = []
y = np.zeros(M)

for j in range(num_of_iterations):
    H = C_op - sum(y[i] * A_ops[i] for i in range(M))

    ansatz = RealAmplitudes(num_qubits=int(np.log2(N)), reps=3)
    optimizer = COBYLA()
    vqe = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
    vqe_result = vqe.compute_minimum_eigenvalue(-H)

    diff_cost_func = np.dot(b, y) + R * max(-vqe_result.eigenvalue.real, 0) - prob.value
    print(f"Step: {j}, How far from actual optimal value: {diff_cost_func}")
    diff_cost_func_iter_store.append(diff_cost_func)

    if vqe_result.eigenvalue.real >= 0:
        y -= learning_rate * b
        continue

    # Use assign_parameters instead of bind_parameters
    optimal_params = dict(zip(ansatz.parameters, vqe_result.optimal_point))
    optimal_circuit = ansatz.assign_parameters(optimal_params)

    expec_A = []
    for A_op in A_ops:
        job = estimator.run(optimal_circuit, A_op)
        expec_A.append(job.result().values[0])

    # Update dual variables
    y -= learning_rate * (b - R * np.array(expec_A))

# Plot convergence
plt.plot(range(num_of_iterations), diff_cost_func_iter_store)
plt.xlabel('Iteration')
plt.ylabel('Difference from optimal value')
plt.title('Convergence of iVQAIC')
plt.show()